In [1]:
import sys
import os
main_path = os.path.abspath(os.path.join(os.path.dirname("./ibopf_pipeline.ipynb"), '..'))
sys.path.insert(0, main_path)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import avocado
import time
from scipy import sparse
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_selection import f_classif, SelectKBest, VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.base import TransformerMixin, BaseEstimator

from src.preprocesing import gen_dataset, gen_dataset_from_h5
from src.transformation.document_transformer import DocumentGeneration, MPDocumentGenerator, DocumentSelector
from src.segmentation.window_slider import TwoWaysSlider
from src.transformation.count_words import count_words, merge_documents, Vectorizer

In [3]:
merged_labels_to_num = {
    "Single microlens": 1,
    "TDE": 2,
    "Short period VS": 3,
    "SN": 4,
    "M-dwarf": 5,
    "AGN": 6,
    "Unknown": 99
}

merged_labels = {
    6: "Single microlens",
    15: "TDE",
    16: "Short period VS",
    42: "SN",
    52: "SN",
    53: "Short period VS",
    62: "SN",
    64: "SN",
    65: "M-dwarf",
    67: "SN",
    88: "AGN",
    90: "SN",
    92: "Short period VS",
    95: "SN",
    99: "Unknown"
}

res, labels, metadata = gen_dataset_from_h5("plasticc_balanced_combined_classes_small_ddf")
bands = ["lsstg", "lssti", "lsstr", "lsstu", "lssty", "lsstz"]
spatial_comp = np.sum([len(ts.observations["flux"]) * 2 for ts in res])
time_durations = np.array([ts.observations["time"].to_numpy()[-1] - ts.observations["time"].to_numpy()[0] for ts in res])
mean_time = np.mean(time_durations)
std_time = np.std(time_durations)
print(mean_time, std_time)

k = spatial_comp // len(res)
print("target k:", k)
labels_merged = np.array([merged_labels_to_num[merged_labels[x]] for x in labels])
print("classes:", np.unique(labels_merged))

Object: 100%|████████████████████████████████████████████████████████████████████| 600/600 [00:00<00:00, 300918.62it/s]

601.5589505662784 299.43984508617797
target k: 463
classes: [1 2 3 4 5 6]


In [22]:
win = 56
wl = 1
alph_size = [4, 4, 4]
quantity = ["mean", "trend", "std"]
index_based_paa = False
num_reduction = False
tol = 4
mean_bp_dist = "normal"
threshold_missing = None
verbose= False

doc_gen = DocumentGeneration(win, word_length=wl, alph_size=alph_size,
                            quantity=quantity, num_reduction=num_reduction,
                            index_based_paa=index_based_paa, tol=tol,
                            mean_bp_dist=mean_bp_dist, threshold_missing=threshold_missing)

In [21]:
doc_gen.bop_size, ((4*4*4)+1) ** 1

(65, 65)

In [5]:
vec = Vectorizer(doc_gen.bop_size, bands)

In [23]:
corpus0 = doc_gen.transform(res)

[win: 56.000000.3, wl: 1, faileds: 0]: 100%|█████████████████████████████████████████| 600/600 [01:00<00:00,  9.99it/s]


In [9]:
sparse.csr_matrix(vec.transform(corpus))

NameError: name 'vec' is not defined

# Precomputing vectorizer in sparse form



In [22]:
limit = int(len(labels) * 0.05)
if __name__ == "__main__":
    wls = [2, 3, 4, 5, 6]
    wins = (mean_time + std_time) * 10 ** np.linspace(-1.95, 0, 40)
    values = []
    out_wins = []
    out_wls = []
    ini = time.time()
    for wl in wls:
        for win in wins:
            mp_doc_gen = MPDocumentGenerator(bands, win=win, n_jobs=6, word_length=wl, alph_size=alph_size,
                            quantity=quantity, num_reduction=num_reduction,
                            index_based_paa=index_based_paa, tol=wl,
                            mean_bp_dist=mean_bp_dist, threshold_missing=threshold_missing)
            vec = Vectorizer(mp_doc_gen.get_bop_size(), bands)
            corpus = np.array(mp_doc_gen.transform(res))
            fails = 0
            for c in corpus:
                if c is None:
                    fails += 1
                    
            if fails > limit:
                print("%d>%s time series failed to be represented, dropping sequence" % (fails, limit))
                continue
            matrix = sparse.csr_matrix(vec.transform(corpus))
            values.append(matrix)
            out_wins.append(win)
            out_wls.append(wl)
#             if len(values) == 5:
#                 break
#         if len(values) == 5:
#             break
    end = time.time()
    print("TOTAL TIME:", end-ini)


76>30 time series failed to be represented, dropping sequence



69>30 time series failed to be represented, dropping sequence



67>30 time series failed to be represented, dropping sequence



70>30 time series failed to be represented, dropping sequence



63>30 time series failed to be represented, dropping sequence


...


92>30 time series failed to be represented, dropping sequence



111>30 time series failed to be represented, dropping sequence



126>30 time series failed to be represented, dropping sequence



148>30 time series failed to be represented, dropping sequence



166>30 time series failed to be represented, dropping sequence
TOTAL TIME: 1670.794047832489


In [54]:
6 * (5 ** 2)

150

In [ ]:
# merge values by class
class_values = []
n_classes = len(np.unique(labels_merged))
classes = np.unique(labels_merged)
for v in values:
    n, m = v.shape
    merged_matrix = np.zeros((n_classes, m), dtype=float)
    for i in range(n):
        label = labels_merged[i]
        k = np.where(label == classes)[0][0]
        merged_matrix[k] += v[i]
    class_values.append(sparse.csr_matrix(merged_matrix))
    
class ClassVectorizer(TransformerMixin):
    def __init__(self):
        self.classes = None
        self.n_classes = None
    
    def fit(self, x, y=None, **kwargs):
        self.classes = np.unique(y)
        self.n_classes = len(self.classes)
        self.k = [np.where(label == self.classes)[0][0] for label in y]
    
    def transform(x, **kwargs):
        class_values = []
        for v in values:
            n, m = v.shape
            merged_matrix = np.zeros((self.n_classes, m), dtype=float)
            for i in range(n):
                k = self.k[i]
                merged_matrix[k] += v[i]
            class_values.append(sparse.csr_matrix(merged_matrix))
        return class_values

In [62]:
# pipeline

norm = "l2"  # cosine normalization
use_idf = True  # compute IDF vector
smooth_idf = True  # prevent zero division, add bias
sublinear_tf = True  # use log-tf
vsm = TfidfTransformer(norm=norm, use_idf=use_idf, smooth_idf=smooth_idf, sublinear_tf=sublinear_tf)

# selector
select = DocumentSelector(idx=-1, data=values, win_arr=out_wins, wl_arr=out_wls)

# reducer
reducer1 = VarianceThreshold()
k = 149
reducer2 = SelectKBest(f_classif, k=k)
# reducer2 = TruncatedSVD(k)

# normalizer
normalizer = Normalizer()

# classifier
classifier = KNeighborsClassifier(n_neighbors=1)


pipe = [
    ("sel", select),
    ("vsm", vsm),
#     ("zeroRed", reducer1),
    ("feaRed", reducer2),
    ("norm", normalizer),
    ("classif", classifier)
]

pipeline = Pipeline(pipe)

In [ ]:
parameters = {
    "sel__idx": np.arange(len(values)),
}

x = np.arange(len(res))
if __name__ == "__main__":
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=6, verbose=1, cv=10)
    t0 = time.time()
    grid_search.fit(x, labels_merged)
    print("done in %0.3fs" % (time.time() - t0))
    

In [59]:
best_acc = grid_search.best_score_
best_idx = grid_search.best_params_
print(best_acc, best_idx)

0.46333333333333326 {'sel__idx': 75}


In [18]:
scores = cross_val_score(pipeline, res, labels_merged, cv=10, scoring='balanced_accuracy')

Object:   0%|                                                                                   | 0/60 [00:00<?, ?it/s]

1 TIME SEIRES FAILED TO BE TRANSFORMED [WIN=100.000000.3, WL=4]


Object:   0%|                                                                                   | 0/60 [00:00<?, ?it/s]

1 TIME SEIRES FAILED TO BE TRANSFORMED [WIN=100.000000.3, WL=4]


Object:   0%|                                                                                   | 0/60 [00:00<?, ?it/s]

...

Object:   0%|                                                                                   | 0/60 [00:00<?, ?it/s]

1 TIME SEIRES FAILED TO BE TRANSFORMED [WIN=100.000000.3, WL=4]


Object:   3%|██▌                                                                        | 2/60 [00:00<00:03, 15.31it/s]

1 TIME SEIRES FAILED TO BE TRANSFORMED [WIN=100.000000.3, WL=4]


Object: 100%|██████████████████████████████████████████████████████████████████████████| 60/60 [00:02<00:00, 22.94it/s]


In [21]:
np.max(scores)

0.4166666666666667

3.229052643315694